

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/NER_TUMOR_ES.ipynb)




# **Detect tumor morphology in Spanish text**

## 1. Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

## 3. Select the DL model

In [ ]:
# If you change the model, re-run all the cells below.
# Applicable models: cantemist_scielowiki
MODEL_NAME = "cantemist_scielowiki"

## 4. Some sample examples

In [ ]:
# Enter examples to be transformed as strings in this list
text_list = [
    """Anamnesis Paciente de 37 años de edad sin antecedentes patológicos ni quirúrgicos de interés. 
    En diciembre de 2012 consultó al Servicio de Urgencias por un cuadro de cefalea aguda e hipostesia del hemicuerpo izquierdo de 15 días de evolución 
    refractario a tratamiento. Exploración neurológica sin focalidad; fondo de ojo: papiledema unilateral. 
    Se solicitaron una TC del SNC, que objetiva una LOE frontal derecha con afectación aparente del cuerpo calloso, y una RM del SNC, 
    que muestra un extenso proceso expansivo intraparenquimatoso frontal derecho que infiltra la rodilla del cuerpo calloso, mal delimitada y sin componente necrótico. 
    Tras la administración de contraste se apreciaban diferentes realces parcheados en la lesión, 
    pero sin definirse una cápsula con aumento del flujo sanguíneo en la lesión, características compatibles con linfoma o astrocitoma anaplásico . 
    El 3 de enero de 2013 se efectúa biopsia intraoperatoria, con diagnóstico histológico de astrocitoma anaplásico GIII"""
]

## 5. Define Spark NLP pipeline

In [ ]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = nlp.Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# Embeddings needs to be the same as the one used to train the model
embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_scielowiki_300d", "es", "clinical/models")\
    .setInputCols("document", "token") \
    .setOutputCol("embeddings")

ner_model = medical.NerModel.pretrained(MODEL_NAME, 'es', 'clinical/models') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = nlp.NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[
        document_assembler, 
        tokenizer,
        embeddings,
        ner_model,
        ner_converter
        ])

embeddings_scielowiki_300d download started this may take some time.
Approximate size to download 351.2 MB
[OK!]
cantemist_scielowiki download started this may take some time.
[OK!]


## 6. Run the pipeline

In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlp_pipeline.fit(df).transform(df)

## 7. Visualize results

In [ ]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)